In [1]:
import pandas as pd
import os
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from scipy.cluster.hierarchy import dendrogram, linkage

#read in the file using pandas an encoding of ISO-8859-1
df = pd.read_csv("https://raw.githubusercontent.com/ggotoo/Artificial-Intelligence/main/AI_Condensed_Data_Useful_Data.csv")

In [3]:
#Check to see if there are any missing values
df.isnull().sum()/100

QKEY                  0.00
TECH1_W99            51.07
CNCEXC_W99            0.00
EXCITEOE_W99_OE1     84.07
EXCITEOE_W99_OE2     99.23
EXCITEOE_W99_OE3    102.01
ALGFAIR_W99           0.00
DISCRIM1_a_W99        0.00
DISCRIM1_b_W99        0.00
DISCRIM1_c_W99        0.00
DISCRIM1_d_W99        0.00
DISCRIM1_e_W99        0.00
DISCRIM1_f_W99        0.00
F_AGECAT              0.00
F_GENDER              0.00
F_EDUCCAT2            0.00
F_RACETHNMOD          0.00
F_CITIZEN             0.00
DCARS1_W99           51.07
DCARS2_W99           51.07
DCARS3_W99           51.07
DCARS4_a_W99         51.07
DCARS4_b_W99         51.07
DCARS4_c_W99         51.07
DCARS4_d_W99         51.07
DCARS5_W99           51.07
DCARS6_W99           51.07
DCARS7_W99           51.07
DCARS8_a_W99         51.07
DCARS8_b_W99         51.07
DCARS8_c_W99         51.07
DCARS8_d_W99         51.07
DCARS9_W99           51.07
DCARS10_W99          51.07
DCARS11_a_W99        51.07
DCARS11_b_W99        51.07
DCARS11_c_W99        51.07
D

In [4]:
# Display the shape of the dataframe
print(df.shape)
# Display the data types of the dataframe
print(df.dtypes)

(10260, 48)
QKEY                float64
TECH1_W99            object
CNCEXC_W99           object
EXCITEOE_W99_OE1     object
EXCITEOE_W99_OE2     object
EXCITEOE_W99_OE3     object
ALGFAIR_W99          object
DISCRIM1_a_W99       object
DISCRIM1_b_W99       object
DISCRIM1_c_W99       object
DISCRIM1_d_W99       object
DISCRIM1_e_W99       object
DISCRIM1_f_W99       object
F_AGECAT             object
F_GENDER             object
F_EDUCCAT2           object
F_RACETHNMOD         object
F_CITIZEN            object
DCARS1_W99           object
DCARS2_W99           object
DCARS3_W99           object
DCARS4_a_W99         object
DCARS4_b_W99         object
DCARS4_c_W99         object
DCARS4_d_W99         object
DCARS5_W99           object
DCARS6_W99           object
DCARS7_W99           object
DCARS8_a_W99         object
DCARS8_b_W99         object
DCARS8_c_W99         object
DCARS8_d_W99         object
DCARS9_W99           object
DCARS10_W99          object
DCARS11_a_W99        object
DCARS11_

In [5]:
#first I will inspect the three columns to see if they can be merged together
columns_to_inspect = ['EXCITEOE_W99_OE3', 'EXCITEOE_W99_OE2', 'EXCITEOE_W99_OE1']

print("First few rows of the columns to inspect:")
print(df[columns_to_inspect].head())

print("Information about the columns:")
print(df[columns_to_inspect].info())

# Check for unique values in each column
for column in columns_to_inspect:
    print(f"Unique values in {column}:")
    print(df[column].unique())

# Check for missing values in each column
print("Missing values in each column:")
print(df[columns_to_inspect].isnull().sum())

# Check if merging the columns can help fill in the gaps
merged_column = df[columns_to_inspect].bfill(axis=1).iloc[:, 0]
print("Merged column with backfill (first few rows):")
print(merged_column.head())

# Merge the columns using backfill (or forward fill if appropriate)
df['Merged_EXCITOE_W99'] = df[columns_to_inspect].bfill(axis=1).iloc[:, 0]

# Display the merged column
print("First few rows of the merged column:")
print(df[['Merged_EXCITOE_W99']].head())

First few rows of the columns to inspect:
  EXCITEOE_W99_OE3 EXCITEOE_W99_OE2      EXCITEOE_W99_OE1
0              NaN              NaN                   NaN
1              NaN              NaN  Don't know/No answer
2              NaN              NaN                   NaN
3              NaN              NaN                   NaN
4              NaN              NaN                   NaN
Information about the columns:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10260 entries, 0 to 10259
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   EXCITEOE_W99_OE3  59 non-null     object
 1   EXCITEOE_W99_OE2  337 non-null    object
 2   EXCITEOE_W99_OE1  1853 non-null   object
dtypes: object(3)
memory usage: 240.6+ KB
None
Unique values in EXCITEOE_W99_OE3:
[nan 'Helps with work/labor' 'Makes life, society better'
 'Handles mundane, tedious tasks'
 'Helps those who are elderly/have a disability' 'Personal anecdote

In [6]:
df.head()

,QKEY,TECH1_W99,CNCEXC_W99,EXCITEOE_W99_OE1,EXCITEOE_W99_OE2,EXCITEOE_W99_OE3,ALGFAIR_W99,DISCRIM1_a_W99,DISCRIM1_b_W99,DISCRIM1_c_W99,...,DCARS13_a_W99,DCARS13_b_W99,DCARS13_c_W99,DCARS13_d_W99,F_MARITAL,F_RELIG,F_PARTY_FINAL,F_REG,F_INC_TIER2,Merged_EXCITOE_W99
0,100260.0,NaN,Equally concerned and excited,NaN,NaN,NaN,Not possible,Not too well,Very well,Very well,...,NaN,NaN,NaN,NaN,Never been married,Nothing in particular,Republican,You are ABSOLUTELY CERTAIN that you are regist...,Middle income,NaN
1,100314.0,Mostly positive,More excited than concerned,Don't know/No answer,NaN,NaN,Possible,Very well,Not too well,Not too well,...,Not sure,Oppose,Favor,Favor,Divorced,Protestant,Republican,You are ABSOLUTELY CERTAIN that you are regist...,Middle income,Don't know/No answer
2,100363.0,Mostly positive,Equally concerned and excited,NaN,NaN,NaN,Not possible,Somewhat well,Somewhat well,Somewhat well,...,Favor,Not sure,Favor,Favor,Married,Roman Catholic,Democrat,You are ABSOLUTELY CERTAIN that you are regist...,Upper income,NaN
3,100598.0,NaN,Equally concerned and excited,NaN,NaN,NaN,Not possible,Not sure,Not sure,Not sure,...,NaN,NaN,NaN,NaN,Married,Protestant,Republican,You are ABSOLUTELY CERTAIN that you are regist...,Upper income,NaN
4,100637.0,NaN,Equally concerned and excited,NaN,NaN,NaN,Not sure,Not sure,Not sure,Not sure,...,NaN,NaN,NaN,NaN,Married,Protestant,Independent,You are ABSOLUTELY CERTAIN that you are regist...,Lower income,NaN


In [10]:
#Drop the colums after merge
df_dropped = df.drop(["EXCITEOE_W99_OE1","EXCITEOE_W99_OE2","EXCITEOE_W99_OE3"], axis=1)

In [9]:
df_dropped.head()

,QKEY,TECH1_W99,CNCEXC_W99,ALGFAIR_W99,DISCRIM1_a_W99,DISCRIM1_b_W99,DISCRIM1_c_W99,DISCRIM1_d_W99,DISCRIM1_e_W99,DISCRIM1_f_W99,...,DCARS13_a_W99,DCARS13_b_W99,DCARS13_c_W99,DCARS13_d_W99,F_MARITAL,F_RELIG,F_PARTY_FINAL,F_REG,F_INC_TIER2,Merged_EXCITOE_W99
0,100260.0,NaN,Equally concerned and excited,Not possible,Not too well,Very well,Very well,Very well,Not sure,Not sure,...,NaN,NaN,NaN,NaN,Never been married,Nothing in particular,Republican,You are ABSOLUTELY CERTAIN that you are regist...,Middle income,NaN
1,100314.0,Mostly positive,More excited than concerned,Possible,Very well,Not too well,Not too well,Not sure,Somewhat well,Very well,...,Not sure,Oppose,Favor,Favor,Divorced,Protestant,Republican,You are ABSOLUTELY CERTAIN that you are regist...,Middle income,Don't know/No answer
2,100363.0,Mostly positive,Equally concerned and excited,Not possible,Somewhat well,Somewhat well,Somewhat well,Somewhat well,Very well,Somewhat well,...,Favor,Not sure,Favor,Favor,Married,Roman Catholic,Democrat,You are ABSOLUTELY CERTAIN that you are regist...,Upper income,NaN
3,100598.0,NaN,Equally concerned and excited,Not possible,Not sure,Not sure,Not sure,Not sure,Not sure,Not sure,...,NaN,NaN,NaN,NaN,Married,Protestant,Republican,You are ABSOLUTELY CERTAIN that you are regist...,Upper income,NaN
4,100637.0,NaN,Equally concerned and excited,Not sure,Not sure,Not sure,Not sure,Not sure,Very well,Not sure,...,NaN,NaN,NaN,NaN,Married,Protestant,Independent,You are ABSOLUTELY CERTAIN that you are regist...,Lower income,NaN


In [12]:
#Check the MISSING VALUES IN THE DATASET for the new dataset
df_dropped.isnull().sum()/100

QKEY                   0.00
TECH1_W99             51.07
CNCEXC_W99             0.00
ALGFAIR_W99            0.00
DISCRIM1_a_W99         0.00
DISCRIM1_b_W99         0.00
DISCRIM1_c_W99         0.00
DISCRIM1_d_W99         0.00
DISCRIM1_e_W99         0.00
DISCRIM1_f_W99         0.00
F_AGECAT               0.00
F_GENDER               0.00
F_EDUCCAT2             0.00
F_RACETHNMOD           0.00
F_CITIZEN              0.00
DCARS1_W99            51.07
DCARS2_W99            51.07
DCARS3_W99            51.07
DCARS4_a_W99          51.07
DCARS4_b_W99          51.07
DCARS4_c_W99          51.07
DCARS4_d_W99          51.07
DCARS5_W99            51.07
DCARS6_W99            51.07
DCARS7_W99            51.07
DCARS8_a_W99          51.07
DCARS8_b_W99          51.07
DCARS8_c_W99          51.07
DCARS8_d_W99          51.07
DCARS9_W99            51.07
DCARS10_W99           51.07
DCARS11_a_W99         51.07
DCARS11_b_W99         51.07
DCARS11_c_W99         51.07
DCARS11_d_W99         51.07
DCARS12_W99         

In [13]:
df_dropped = df.fillna(np.mean())

TypeError: mean() missing 1 required positional argument: 'a'